In [1]:
# Some path wizardry to make python acknowledge relative paths, just ignore this part...
import sys; sys.path.append('..')

In [2]:
import networkx as nx
from pelote import to_nodes_dataframe

In [3]:
g = nx.read_gexf('./data/rio.gexf')

In [7]:
to_nodes_dataframe(g, node_key_col='node')

,node,Label2,Nature of institution,Category,Latest Six Posts Frequency,Language,Information Resources,viz,label
0,ncs1,uncsd2012.org,Event,Green-economy,Dially,English,Multi-Wiki,"{'color': {'r': 197, 'g': 91, 'b': 54, 'a': 1....",http://www.uncsd2012.org
1,ncs2,un.org,Trans-institutional,Green-economy,Undefined,All,No,"{'color': {'r': 197, 'g': 91, 'b': 54, 'a': 1....",http://www.un.org
2,ncs3,rio20.info,Event,Green-economy,Dially,Portuguese,Multimedia,"{'color': {'r': 197, 'g': 91, 'b': 54, 'a': 1....",http://www.rio20.info
3,ncs4,cupuladospovos.org.br,Event,Social Ecology,Dially,Portuguese,Multimedia,"{'color': {'r': 144, 'g': 107, 'b': 198, 'a': ...",http://cupuladospovos.org.br
4,ncs5,fase.org.br,NGO,Social Ecology,Undefined,Portuguese,No,"{'color': {'r': 144, 'g': 107, 'b': 198, 'a': ...",http://www.fase.org.br
...,...,...,...,...,...,...,...,...,...
361,ncs387,econsciencia.org.br,NGO,Deep Ecology,Undefined,Portuguese,Multimedia,"{'color': {'r': 103, 'g': 143, 'b': 57, 'a': 1...",http://www.econsciencia.org.br
362,ncs320,desdemonadespair.net,Individual,Deep Ecology,Dially,English,Multimedia,"{'color': {'r': 103, 'g': 143, 'b': 57, 'a': 1...",http://www.desdemonadespair.net
363,ncs394,igre.org.br,NGO,Social Ecology,Undefined,Portuguese,Multimedia,"{'color': {'r': 144, 'g': 107, 'b': 198, 'a': ...",http://www.igre.org.br
364,ncs385,inga.org.br,Educational,Social Ecology,Undefined,Portuguese,Multimedia,"{'color': {'r': 144, 'g': 107, 'b': 198, 'a': ...",http://www.inga.org.br
